# Time series

## Import data and libraries

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import geopandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
#evict_hist = pd.read_excel("Data/2022_Eviction_Data/DallasCounty_HistoricalEvictionRecords.xlsx")
#evict_hist.head(2)

In [ ]:
evict_hist.columns

# Remove businesses

In [ ]:
def def_names(df):
    df['def_business'] = 0
        
    # Sort out defendent names containing a key word
    key_words = [' INC', ' LLC', 'SHOP', ' LLP', 'RESTAURANT', 'PARTNER', 'CORP']
    for key in key_words:
        df.loc[df['defendant_name'].str.contains(key), 'def_business'] = 1
    
    return(df)

In [ ]:
evict_hist_def_names = def_names(evict_hist)
evict_hist_no_bus = evict_hist_def_names[evict_hist_def_names['def_business'] == 0]
len(evict_hist_no_bus)

# Extract years and months

In [ ]:
# Extract yr-mo
evict_yr_mo = []
evict_mo = []
for i in evict_hist_no_bus['filed_date'].astype(str):
    if len(i.split('-')) > 2:
        evict_yr_mo.append(i.split('-')[0] + '-' + i.split('-')[1])
        evict_mo.append(i.split('-')[1])
    else:
        evict_yr_mo.append(i.split('-')[0])
        evict_mo.append('Na')

In [ ]:
# Add yr-mo and mo to data frame
evict_hist_no_bus['yr_mo'] = evict_yr_mo

# Find number of evictions over time

In [ ]:
# Group by yr_mo
evict_hist_yr_mo_group = evict_hist_no_bus.groupby('yr_mo').nunique()
evict_hist_yr_mo_group = evict_hist_yr_mo_group.drop(index=['2011-11','NaT'])
evict_hist_yr_mo_group['yr_mo_col'] = evict_hist_yr_mo_group.index
evict_hist_yr_mo_group.head(3)


In [ ]:
# Format yr_mo as number
evict_hist_yr_mo_group['yr_mo_col_num'] = evict_hist_yr_mo_group['yr_mo_col'].str.replace('-','').astype(float)

In [ ]:
# Order months to allow for linear regression
evict_hist_yr_mo_group['yr_mo_col_ord'] = range(len(evict_hist_yr_mo_group['yr_mo_col_num']))
evict_hist_yr_mo_group['yr_mo_col_ord']

In [ ]:
# Add variable for month
ev_mo = []
for i in evict_hist_yr_mo_group['yr_mo_col']:
    ev_mo.append(i.split('-')[1])
        
evict_hist_yr_mo_group['ev_mo'] = ev_mo

In [ ]:
# Add variable that denotes if pre/post covid
covid_status = []
for i in evict_hist_yr_mo_group['yr_mo_col_num']:
    if i < 202003:
        covid_status.append('Pre-Covid')
    else:
        covid_status.append('Covid and Post-Covid')
        
evict_hist_yr_mo_group['covid_status'] = covid_status

In [ ]:
plt.style.use('ggplot')
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5), layout='constrained')

# Add scatterplot + legend
pre_col = '#315094'
post_col = '#346732'
cov_colors = {'Pre-Covid': pre_col, 'Covid and Post-Covid': post_col}
scatter = sns.scatterplot(data=evict_hist_yr_mo_group, x='yr_mo_col', y='case_number', palette=cov_colors,
                          ax=ax, hue='covid_status', s=40).set(title='Number of Evictions by Month and Year', 
                                           ylabel='Number of Evictions', xlabel='Year-Month');

ax.legend(loc=4, ncol=1, title='Covid Status')#, labels=['Pre-Covid', 'Covid and Post-Covid'])

# Add annotation by year
yr_alpha = 0.2
text_y = 4250
ax.axvspan('2017-01', '2017-12', color='#B2C2E6', alpha=yr_alpha)
ax.annotate('2017', xy =('2017-06', text_y), ha='left')
ax.axvspan('2018-01', '2018-12', color='#6686CC', alpha=yr_alpha)
ax.annotate('2018', xy =('2018-06', text_y), ha='left')
ax.axvspan('2019-01', '2019-12', color='#315094', alpha=yr_alpha)
ax.annotate('2019', xy =('2019-06', text_y), ha='left')

ax.axvspan('2020-01', '2020-03', color='#315094', alpha=yr_alpha)
ax.axvspan('2020-03', '2020-12', color='#BBDEBA', alpha=yr_alpha)

ax.annotate('2020', xy =('2020-06', text_y), ha='left')
ax.axvspan('2021-01', '2021-12', color='#78BD75', alpha=yr_alpha)
ax.annotate('2021', xy =('2021-06', text_y), ha='left')
ax.axvspan('2022-01', '2022-06', color='#346732', alpha=yr_alpha)
ax.annotate('2022', xy =('2022-03', text_y), ha='left')


# Add moratorium to graph
mora_color = 'red'
ax.axvline('2020-08', color=mora_color, alpha=0.7)
ax.axvline('2021-09', color=mora_color, alpha=0.7)
ax.annotate('------------Eviction Moratorium------------', xy =('2021-09', text_y-1000), ha='right', color=mora_color)

# Add regression plots
pol_order=1
lin_plot = sns.regplot(x ='yr_mo_col_ord', y ='case_number', order=1, color=pre_col,
                       data = evict_hist_yr_mo_group[evict_hist_yr_mo_group['covid_status']=='Pre-Covid'], ax=ax);

lin_plot = sns.regplot(x ='yr_mo_col_ord', y ='case_number', order=1, color=post_col,
                       data = evict_hist_yr_mo_group[evict_hist_yr_mo_group['covid_status']=='Covid and Post-Covid'], ax=ax).set(title='Number of Evictions by Month and Year', 
                                           ylabel='Number of Evictions', xlabel='Year-Month');

# Label freeze
bbox = dict(boxstyle ="round", fc ="0.8")
#arrowprops = dict(
 #   arrowstyle = "->", color='black',
  #  connectionstyle = "angle, angleA = 0, angleB = 90,\
   # rad = 10")
arrowprops=dict(arrowstyle="simple",
                            color='red', ec="none",
                            connectionstyle="angle3,angleA=0,angleB=-90")
ax.annotate('The Great\nTexas Freeze', xy=('2021-02', 498), xytext=(70, -30), 
            textcoords='offset points', ha='center', va='bottom',
            bbox = bbox, arrowprops = arrowprops)

# Adjust x-axis labels
ax.set_xticks(range(len(evict_hist_yr_mo_group)));
ax.set_xticklabels(evict_hist_yr_mo_group['yr_mo_col']);
plt.xticks(rotation=70);

plt.savefig('Results/July5.2023/evict.mo.yr.png', dpi=1200)

# Graph number of evictions by month, pre- and post-covid

In [ ]:
# Extract pre and post covid
pre_evict_hist_yr_mo_group = evict_hist_yr_mo_group[evict_hist_yr_mo_group['covid_status']=='Pre-Covid']
post_evict_hist_yr_mo_group = evict_hist_yr_mo_group[evict_hist_yr_mo_group['covid_status']=='Covid and Post-Covid']

In [ ]:
post_evict_hist_yr_mo_group.columns

In [ ]:
# Find average by month
pre_mo = pre_evict_hist_yr_mo_group.groupby('ev_mo').mean()
post_mo = post_evict_hist_yr_mo_group.groupby('ev_mo').mean()

# Add column for month
pre_mo['mo'] = pre_mo.index
post_mo['mo'] = post_mo.index

In [ ]:
plt.style.use('ggplot')
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), layout='constrained', sharey=True)

pre_bar = sns.barplot(data=pre_mo, x='mo', y='case_number', ax=ax0);
pre_bar.set_title('Pre-Covid: 2017/01 - 2020/03',fontdict={'color': 'grey'})
pre_bar.set_xlabel('Month')
pre_bar.set_ylabel('Number of Evictions')
post_bar = sns.barplot(data=post_mo, x='mo', y='case_number', ax=ax1);
post_bar.set_xlabel('Month')
post_bar.set_ylabel('Number of Evictions')
post_bar.set_title('Covid and Post-Covid: 2020/03 - 2022/06',fontdict={'color': 'grey'})
fig.suptitle('Average Number of Evictions by Month', size=20, ha='center');

#plt.savefig('Results/July5.2023/num_evict_mo_bar.png')